# SoMe content 

## Sustainable finance trend radar
#### 1. Topics that appeared
#### 2. Topics that dropped out
#### 3. Ranks and rank changes

## Extract

In [69]:
%%writefile ../pipelines/analysis_dashboard/extract.py
"""
~-- EXTRACT
Loads the queried data from the raw data directory
"""

from glob import glob
import os
import pandas as pd

def load_data(raw_data_dir, filename):
    # merge all data from directory
    data_files = glob(f'{os.path.join(raw_data_dir, filename)}*')
    df_list = [pd.read_csv(file) for file in data_files]
    df = pd.concat(df_list).reset_index(drop=True)
    
    return df

Overwriting ../pipelines/analysis_dashboard/extract.py


In [68]:
%%writefile ../pipelines/analysis_dashboard/transform.py
"""
~-- TRANSFORM
Create relevant statistics about Google trends data
"""

import pandas as pd

def trends_statistics(df, rank_category='rising'):
    """Generate relevant statistics for a ranking category of Google trends (rising or top)"""
    
    ## subset by Google Trends category: rising or top
    df = df.loc[df.ranking == rank_category].reset_index(drop=True) # select only rising topics

    ## datetime
    # timestamp to date
    df['query_date'] = pd.to_datetime(df.query_timestamp).dt.date
    # time series indicator: t
    df['t'] = df.groupby('query_date').ngroup() 
    # most recent period
    t_max = df.t.max() # most recent

    ## ranking 
    # absolute
    df['rank_t'] = df.groupby('t').value.rank(method='first', ascending=False) 
    df['rank_t-1'] = df.groupby('query').rank_t.shift() # rank in previous period (t-1) 
    df['rank_absolute_change'] = df.rank_t - df['rank_t-1'] # rank change from previous, t-1, to current period, t
    # winners and loosers (ranking of absoulte changes)
    df['rank_absoulte_change_ranking'] =  df.groupby('t').rank_absolute_change.rank(method='first', ascending=False)
    
    # percentile
    df['rank_pct_t'] = df.groupby('t').value.rank(method='first', ascending=False, pct=True)
    df['rank_pct_t-1'] = df.groupby('query').rank_pct_t.shift()
    df['rank_pct_change'] = df.rank_pct_t - df['rank_pct_t-1']    

    ## new entries at time t
    df['new_entry_t'] = (pd.isna(df['rank_t-1']) & pd.notnull(df.rank_t))*1
    
    ## dropouts at time t+1
    # keywords for each period  to compare sets
    queries_dict = df.groupby('t')['query'].apply(list).to_dict()
    # compare query responses sets across last two periods 
    dropouts = list(set(queries_dict[t_max-1]).difference(set(queries_dict[t_max]))) 
    df['dropout_t+1'] = ((df.t == t_max-1) & df['query'].isin(dropouts))*1

    ## fill missings
    df = df.fillna(0)
    
    return df

def dashboard_data(df):
    """Create statistics for each category and concatenate dataframes """
    ranking_categories = df.ranking.unique().tolist()
    df = pd.concat([trends_statistics(df, rank_category=r) for r in ranking_categories])
    
    return df

Overwriting ../pipelines/analysis_dashboard/transform.py


## Development / Testing

In [1]:
from glob import glob
glob('../data/2_final/*')

['../data/2_final\\esg_trends_analysis.csv',
 '../data/2_final\\esg_trends_analysis_210405_1327.csv',
 '../data/2_final\\README.md',
 '../data/2_final\\sustainable_finance.csv']

In [4]:
# %load ../serve/utils/app_utilities.py
import yaml
import os
import pandas as pd

def load_config(filepath):
	"""Return dictionary with settings"""
	with open(filepath) as file:
		config = yaml.full_load(file)
		return config

def load_data(filepath):
	"""Read csv-only file from data_dir/filename"""
	df = pd.read_csv(filepath)
	return df

In [17]:
df = load_data('../data/2_final/esg_trends_analysis.csv').query("ranking == 'rising'")
df.query_date = pd.to_datetime(df.query_date).dt.strftime('%d.%m.%Y')

# -- process data
t_most_recent = df.t.max()-1
# most recent date
date_most_recent = df.loc[df.t==t_most_recent].query_date.unique()[0]

df_selected_date = df.loc[df.query_date==date_most_recent].drop_duplicates(subset=['ranking', 'query', 'query_date'])

df_selected_date.sort_values(by='rank_t').loc[:,['query', 'rank_t', 'rank_absolute_change', 'query_date']].head(10)

,query,rank_t,rank_absolute_change,query_date
3705,sustainable finance action plan,1.0,-33.0,05.04.2021
3706,taxonomy,2.0,0.0,05.04.2021
3707,eu sustainable finance taxonomy,3.0,-2.0,05.04.2021
3708,eu sustainable finance action plan,4.0,0.0,05.04.2021
3691,sdgs,17.0,12.0,05.04.2021
3692,sdgs esg,19.0,15.0,05.04.2021
3709,sustainable finance disclosure regulation,23.0,14.0,05.04.2021
3710,eu action plan on sustainable finance,24.0,3.0,05.04.2021
3711,international platform on sustainable finance,31.0,12.0,05.04.2021
3726,socially responsible investment advisors,32.0,16.0,05.04.2021


,query,value,keyword,ranking,geo,query_timestamp,query_date,t,rank_t,rank_t-1,rank_absolute_change,rank_absoulte_change_ranking,rank_pct_t,rank_pct_t-1,rank_pct_change,new_entry_t,dropout_t+1
3681,best sustainable investment funds,49800,sustainable investment,rising,global,2021-04-05 13:36:31.561587,05.04.2021,23,49.0,29.0,20.0,83.0,0.200820,0.547170,-0.346350,0,1
3682,global sustainable investment review 2018,16600,sustainable investment,rising,global,2021-04-05 13:36:31.561587,05.04.2021,23,96.0,50.0,46.0,68.0,0.393443,0.757576,-0.364133,0,1
3683,sustainable development goals,850,sustainable investment,rising,global,2021-04-05 13:36:31.561587,05.04.2021,23,140.0,35.0,105.0,32.0,0.573770,0.660377,-0.086607,0,1
3684,global sustainable investment review,500,sustainable investment,rising,global,2021-04-05 13:36:31.561587,05.04.2021,23,148.0,39.0,109.0,29.0,0.606557,0.735849,-0.129292,0,0
3685,global sustainable investment alliance,400,sustainable investment,rising,global,2021-04-05 13:36:31.561587,05.04.2021,23,149.0,38.0,111.0,27.0,0.610656,0.716981,-0.106325,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783,sustainable finance eu,2850,sustainable finance,rising,global,2021-04-05 13:37:09.030702,05.04.2021,23,130.0,31.0,99.0,37.0,0.532787,0.584906,-0.052119,0,1
3788,sustainable capital finance,40,sustainable finance,rising,global,2021-04-05 13:37:09.030702,05.04.2021,23,240.0,42.0,198.0,1.0,0.983607,0.736842,0.246764,0,1
3857,pri principles for responsible investment,140,responsible investment,rising,global,2021-04-05 13:38:23.892142,05.04.2021,23,181.0,50.0,131.0,23.0,0.741803,0.877193,-0.135390,0,1
3859,pri,130,responsible investment,rising,global,2021-04-05 13:38:23.892142,05.04.2021,23,185.0,46.0,139.0,18.0,0.758197,0.807018,-0.048821,0,0


In [17]:
df_selected_date.sort_values(by='rank_absolute_change').loc[:,['query', 'rank_t', 'rank_absolute_change']].head(10)

,query,rank_t,rank_absolute_change
3243,environmentally responsible investment,8.0,-35.0
3230,sustainable finance courses,14.0,-27.0
3220,axis esg equity fund regular growth,31.0,-26.0
3231,high-level expert group on sustainable finance,15.0,-18.0
3201,global sustainable investment alliance,38.0,-7.0
3203,sustainable investment fund,44.0,-7.0
3240,swiss sustainable finance,37.0,-6.0
3214,esg stands for,24.0,-6.0
3207,sustainable investment management,50.0,-5.0
3215,kotak esg opportunities fund,25.0,-5.0


## Analyses
### Newcomer 

In [20]:
df_selected_date.loc[(df_selected_date.t == t_most_recent) & (df_selected_date.new_entry_t == 1)]

,query,value,keyword,ranking,geo,query_timestamp,query_date,t,rank_t,rank_t-1,rank_absolute_change,rank_absoulte_change_ranking,rank_pct_t,rank_pct_t-1,rank_pct_change,new_entry_t,dropout_t+1
3219,axis esg equity fund direct growth,5350,ESG,rising,global,2021-04-02 08:31:37.360007,02.04.2021,22,30.0,0.0,0.0,0.0,0.566038,0.0,0.0,1,0


### Dropouts

In [268]:
df.loc[(df.t == t_max-1) & (df['dropout_t+1'] == 1)]

,query,value,keyword,ranking,geo,query_timestamp,query_date,day,month,year,t,rank_t,rank_t-1,rank_change,rank_pct_t,rank_pct_t-1,rank_pct_change,new_entry_t,dropout_t+1
1258,journal of sustainable finance and investment,120,sustainable investment,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,50.0,42.0,8.0,0.877193,0.736842,0.140351,0,1
1260,sri,60,sustainable investment,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,55.0,48.0,7.0,0.964912,0.905660,0.059252,0,1
1263,vanguard esg,47950,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,11.0,18.0,-7.0,0.192982,0.321429,-0.128446,0,1
1265,icici esg fund,24150,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,19.0,19.0,0.0,0.333333,0.358491,-0.025157,0,1
1266,sbi magnum equity esg fund,22100,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,21.0,26.0,-5.0,0.368421,0.490566,-0.122145,0,1
1267,msci esg ratings,19400,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,23.0,NaN,NaN,0.403509,NaN,NaN,1,1
1268,axis esg equity fund nav,14400,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,24.0,30.0,-6.0,0.421053,0.566038,-0.144985,0,1
1269,esg investing funds,12500,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,25.0,NaN,NaN,0.438596,NaN,NaN,1,1
1270,esg investing meaning,12050,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,26.0,29.0,-3.0,0.456140,0.547170,-0.091029,0,1
1271,esg investing companies,9500,ESG,rising,global,2021-04-01 06:40:13.022014,2021-04-01,1,4,2021,21,27.0,33.0,-6.0,0.473684,0.589286,-0.115602,0,1


### Being-listed streak

In [311]:
# count across all t
df['overall_count'] = df.groupby('query').value.transform('count')

In [322]:
df.count()

query              1361
value              1361
keyword            1361
ranking            1361
geo                1361
query_timestamp    1361
query_date         1361
day                1361
month              1361
year               1361
t                  1361
rank_t             1361
rank_t-1           1120
rank_change        1120
rank_pct_t         1361
rank_pct_t-1       1120
rank_pct_change    1120
new_entry_t        1361
dropout_t+1        1361
listing_count         0
overall_count      1361
dtype: int64

In [319]:
df.groupby('query').overall_count.min().sort_values(ascending=False).head(10)

query
sustainable and responsible investment           24
axis esg equity fund                             23
sustainable finance disclosure regulation        23
international platform on sustainable finance    23
global sustainable investment alliance           23
sustainable responsible investment               22
sdgs                                             22
eu action plan on sustainable finance            22
expert panel on sustainable finance              22
global sustainable investment review             21
Name: overall_count, dtype: int64